In [1]:
import os
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm

# Data Download and Preprocessing Functions
def download_data(stocks, market_index, start_date, end_date):
    data = yf.download(stocks + [market_index], start=start_date, end=end_date, interval='1mo')['Adj Close']
    return data

def load_and_clean_factors(mom_file, ff_file):
    mom_factors = pd.read_csv(mom_file, index_col=None)
    mom_factors['Date'] = pd.to_datetime(mom_factors['Date'], format='%Y%m').dt.date

    ff_factors = pd.read_csv(ff_file, index_col=None)
    ff_factors['Date'] = pd.to_datetime(ff_factors['Date'], format='%Y%m').dt.date

    factors_merged = pd.merge(ff_factors, mom_factors, on='Date', how='inner')
    factors_merged['RF'] = ((factors_merged['RF'] / 100) / 12)
    factors_merged['RF'] = np.log(factors_merged['RF'] + 1).dropna()
    return factors_merged

def calculate_log_returns(data):
    data.index = pd.to_datetime(data.index).date
    log_returns = np.log(data / data.shift(1)).dropna()
    return log_returns

def merge_data(log_returns, factors_merged):
    log_returns['Date'] = log_returns.index
    combined_data = pd.merge(log_returns, factors_merged, on='Date', how='inner')
    combined_data.set_index('Date', inplace=True)
    return combined_data

def calculate_risk_premiums(combined_data, stocks):
    stock_risk_premium_data = {}
    for stock in stocks:
        stock_risk_premium = combined_data[stock] - combined_data['RF']
        stock_risk_premium_data[stock] = stock_risk_premium

    stock_risk_premium_df = pd.DataFrame(stock_risk_premium_data)
    market_risk_premium = combined_data['SPY'] - combined_data['RF']
    return stock_risk_premium_df, market_risk_premium

def get_yahoo_betas(stocks):
    comparison = {}
    for stock in stocks:
        comparison[stock] = yf.Ticker(stock).info.get('beta', None)
    return comparison

def capm_analysis(stock_risk_premium_df, market_risk_premium, stocks):
    results = []
    for stock in stocks:
        stock_risk_premium = stock_risk_premium_df[stock]
        X = sm.add_constant(market_risk_premium)
        model = sm.OLS(stock_risk_premium, X).fit()
        alpha, beta = model.params
        alpha_pval = model.pvalues['const']
        results.append({'Stock': stock, 'Alpha': alpha, 'Beta': beta, 'Alpha p-value': alpha_pval})

    yahoo_betas = get_yahoo_betas(stocks)
    capm_results = pd.DataFrame(results)
    capm_results['Yahoo Beta'] = capm_results['Stock'].map(yahoo_betas)
    return capm_results

def fama_french_analysis(stock_risk_premium_df, combined_data, market_risk_premium, stocks):
    results_ff = []
    for stock in stocks:
        stock_risk_premium = stock_risk_premium_df[stock]
        X = sm.add_constant(pd.concat([market_risk_premium, combined_data[['SMB', 'HML', 'Mom']]], axis=1))
        model = sm.OLS(stock_risk_premium, X).fit()
        alpha = model.params['const']
        beta_mkt, beta_smb, beta_hml, beta_mom = model.params[1:]
        alpha_pval = model.pvalues['const']
        residual_ss = model.ssr
        residual_df = model.df_resid
        residual_var = residual_ss / residual_df
        results_ff.append({
            'Stock': stock, 'Alpha': alpha, 'Beta MKT': beta_mkt, 'Beta SMB': beta_smb,
            'Beta HML': beta_hml, 'Beta MOM': beta_mom, 'Alpha p-value': alpha_pval,
            'SSR': residual_ss, 'DF Residual': residual_df, 'Residual Variance': residual_var
        })

    yahoo_betas = get_yahoo_betas(stocks)
    ff_results = pd.DataFrame(results_ff)
    ff_results['Yahoo Beta'] = ff_results['Stock'].map(yahoo_betas)
    return ff_results

def compute_tangent_portfolio_weights(diff_returns_rf, cov_matrix):
    """
    Computes the tangent portfolio weights.

    Parameters:
    - diff_returns_rf: Excess returns over the risk-free rate
    - cov_matrix: Variance-covariance matrix of stock returns

    Returns:
    - Tangent portfolio weights as a numpy array
    """
    excess_returns = diff_returns_rf.to_numpy()
    cov_matrix_inv = np.linalg.inv(cov_matrix)
    tangent_numerator = np.dot(cov_matrix_inv, excess_returns)
    tangent_denominator = np.dot(np.ones(len(excess_returns)), tangent_numerator)
    tangent_weights = tangent_numerator / tangent_denominator
    return tangent_weights

def compute_sharpe_ratio_tangent_portfolio(tangent_weights, diff_returns_rf, cov_matrix):
    """
    Computes the Sharpe ratio of the tangent portfolio.

    Parameters:
    - tangent_weights: Weights of the tangent portfolio
    - diff_returns_rf: Excess returns over the risk-free rate
    - cov_matrix: Variance-covariance matrix of stock returns

    Returns:
    - Sharpe ratio of the tangent portfolio
    """
    tangent_port_risk_premium = np.sum(tangent_weights * diff_returns_rf)
    portfolio_variance = np.dot(tangent_weights.T, np.dot(cov_matrix, tangent_weights))
    portfolio_std_dev = np.sqrt(portfolio_variance)
    sharpe_ratio_tangent_port = tangent_port_risk_premium / portfolio_std_dev
    return sharpe_ratio_tangent_port

def compute_sharpe_ratio_sim(ff_results, market_risk_premium, combined_data):
    """
    Computes the Sharpe ratio of the SIM portfolio.

    Parameters:
    - ff_results: Results from the Fama-French analysis
    - market_risk_premium: Market risk premium series
    - combined_data: Combined dataset including returns and factors

    Returns:
    - Sharpe ratio of the SIM portfolio
    """
    initial_position = ff_results['Alpha'] / ff_results['Residual Variance']
    scaled_initial_position = initial_position / initial_position.sum()
    weighted_avg_alpha = np.average(ff_results['Alpha'], weights=scaled_initial_position)
    weighted_residual_variance = np.average(ff_results['Residual Variance'], weights=scaled_initial_position**2)

    sharpe_ratio_market = np.mean(market_risk_premium) / np.std(combined_data['SPY'])
    information_ratio_squared = (weighted_avg_alpha ** 2) / weighted_residual_variance
    sharpe_ratio_sim = np.sqrt(sharpe_ratio_market ** 2 + information_ratio_squared)
    return sharpe_ratio_sim


def compute_adjusted_tangent_weights(initial_margin, tanget_stocks, diff_returns_rf, cov_matrix, investment_amount):
    """
    Adjust tangent weights for the given initial margin and investment amount.
    """
    inverse_initial_margin = np.where(initial_margin != 0, 1 / initial_margin, 0)
    identity_matrix = np.eye(len(tanget_stocks))

    tanget_stocks = pd.Series(tanget_stocks)
    inverse_initial_margin = pd.Series(inverse_initial_margin.flatten())
    one_over_k_sign = inverse_initial_margin * np.sign(tanget_stocks)

    one_over_k_sign = one_over_k_sign.to_numpy()
    A_1 = identity_matrix * one_over_k_sign
    A_1[A_1 == -0.0] = 0.0
    tanget_stocks = -tanget_stocks.to_numpy().reshape(-1, 1)

    A = np.hstack((A_1, tanget_stocks))
    new_row = np.append(np.ones(10), [0])
    A = np.vstack((A, new_row))
    c = np.zeros((len(tanget_stocks) + 1, 1))
    c[-1, 0] = investment_amount

    Z = np.matmul(np.linalg.inv(A), c)
    stock_value = (Z[:len(tanget_stocks)] / initial_margin) * np.sign(-tanget_stocks)

    tangent_weights = stock_value / np.sum(stock_value)
    return tangent_weights.flatten()

In [2]:
# Main Execution
stocks = sorted(['AAPL', 'BRK-B', 'COST', 'FIX', 'JNJ', 'JPM', 'LMT', 'NVDA', 'TSLA', 'XOM'])
market_index = 'SPY'
start_date = '2019-09-01'
end_date = '2024-10-01'

data = download_data(stocks, market_index, start_date, end_date)
factors_merged = load_and_clean_factors('F-F_Momentum_Factor.CSV', 'F-F_Research_Data_5_Factors_2x3.CSV')
log_returns = calculate_log_returns(data)
log_returns_stock = log_returns.drop(columns=['SPY'])
combined_data = merge_data(log_returns, factors_merged)
stock_risk_premium_df, market_risk_premium = calculate_risk_premiums(combined_data, stocks)

capm_results = capm_analysis(stock_risk_premium_df, market_risk_premium, stocks)
ff_results = fama_french_analysis(stock_risk_premium_df, combined_data, market_risk_premium, stocks)

# SIM Portfolio Sharpe Ratio
sharpe_ratio_sim = compute_sharpe_ratio_sim(ff_results, market_risk_premium, combined_data)

# Tangent Portfolio
diff_returns_rf = (combined_data[stocks].mean() - combined_data['RF'].mean())
cov_matrix = log_returns_stock.cov().to_numpy()
tanget_stocks = compute_tangent_portfolio_weights(diff_returns_rf, cov_matrix)
sharpe_ratio_tangent = compute_sharpe_ratio_tangent_portfolio(tanget_stocks, diff_returns_rf.to_numpy(), cov_matrix)

# Investor wants to allocate $10,000 across stocks
# initial margin=50% for all assets
initial_margin = np.full((len(tanget_stocks), 1), 0.5)
tangent_weights = compute_adjusted_tangent_weights(initial_margin, tanget_stocks, diff_returns_rf, cov_matrix, 10000)
sharpe_ratio_tangent_1 = compute_sharpe_ratio_tangent_portfolio(tangent_weights, diff_returns_rf.to_numpy(), cov_matrix)

# Investor does not get a margin loan for long positions, but does need to get one for shorting (set it at 50%)
# Recompute the SR
initial_margin = np.where(tanget_stocks < 0, 0.5, 1).reshape(-1, 1)
tangent_weights = compute_adjusted_tangent_weights(initial_margin, tanget_stocks, diff_returns_rf, cov_matrix, 10000)
sharpe_ratio_tangent_2 = compute_sharpe_ratio_tangent_portfolio(tangent_weights, diff_returns_rf.to_numpy(), cov_matrix)

#  Investor sets the initial margin at 75% for any position, even though the requirement is only 50%. 
initial_margin = np.full((10, 1), 0.75)
tangent_weights = compute_adjusted_tangent_weights(initial_margin, tanget_stocks, diff_returns_rf, cov_matrix, 10000)
sharpe_ratio_tangent_3 = compute_sharpe_ratio_tangent_portfolio(tangent_weights, diff_returns_rf.to_numpy(), cov_matrix)

[*********************100%***********************]  11 of 11 completed


In [3]:
# Display Results
print("CAPM Results:\n", capm_results)

CAPM Results:
    Stock     Alpha      Beta  Alpha p-value  Yahoo Beta
0   AAPL  0.009998  1.194633       0.164158       1.240
1  BRK-B  0.002404  0.858080       0.627024       0.871
2   COST  0.010316  0.761937       0.123997       0.789
3    FIX  0.023830  1.066397       0.030239       1.130
4    JNJ -0.000875  0.480121       0.880213       0.518
5    JPM -0.001350  1.116841       0.849944       1.091
6    LMT  0.002275  0.448992       0.777999       0.481
7   NVDA  0.036599  1.643940       0.014415       1.657
8   TSLA  0.023145  2.069860       0.305774       2.295
9    XOM  0.001576  0.872148       0.888763       0.880


In [4]:
print("Fama-French Results:\n", ff_results)

Fama-French Results:
    Stock     Alpha  Beta MKT  Beta SMB  Beta HML      Beta MOM  Alpha p-value  \
0   AAPL  0.009608  1.225489  0.000162 -0.005288  1.455348e-04       0.145391   
1  BRK-B  0.001264  0.926084 -0.003263  0.004050  1.273654e-03       0.773416   
2   COST  0.009776  0.789633 -0.001832 -0.004333 -7.014866e-04       0.113690   
3    FIX  0.022317  1.220920  0.004027  0.001584  6.841886e-03       0.038889   
4    JNJ -0.001957  0.569617  0.000176  0.001835  3.132780e-03       0.731809   
5    JPM -0.000119  1.061681  0.005349  0.006262  1.778981e-03       0.981896   
6    LMT  0.001381  0.502465 -0.002669  0.005845  1.395131e-03       0.854872   
7   NVDA  0.035367  1.728680 -0.001327 -0.012059  8.573501e-07       0.007266   
8   TSLA  0.026976  1.876632  0.015636 -0.017640 -1.736804e-03       0.190960   
9    XOM  0.003010  0.760188 -0.000019  0.011159 -1.599063e-03       0.737818   

        SSR  DF Residual  Residual Variance  Yahoo Beta  
0  0.132797         55.0    

In [5]:
print("Sharpe Ratio of SIM Portfolio:", sharpe_ratio_sim)

Sharpe Ratio of SIM Portfolio: 0.3290651191630374


In [6]:
print("Tangent Portfolio Weights:\n", tangent_weights)

Tangent Portfolio Weights:
 [-0.19280682 -0.69631946  0.27484156  0.46438779  0.46044579 -0.4137363
  0.27341325  0.46168733  0.04404908  0.32403777]


In [7]:
print("Sharpe Ratio of Tangent Portfolio:", sharpe_ratio_tangent)

Sharpe Ratio of Tangent Portfolio: 0.5025648260058019


In [8]:
print("Sharpe Ratio of Tangent Portfolio 1:", sharpe_ratio_tangent_1)

Sharpe Ratio of Tangent Portfolio 1: 0.5025648260058019


In [9]:
print("Sharpe Ratio of Tangent Portfolio 2:", sharpe_ratio_tangent_2)

Sharpe Ratio of Tangent Portfolio 2: 0.5025648260058019


In [10]:
print("Sharpe Ratio of Tangent Portfolio 3:", sharpe_ratio_tangent_3)

Sharpe Ratio of Tangent Portfolio 3: 0.502564826005802
